# Basic Instructions 
- **1)** Run the script.
- **2)** An interactive visualization module will appear at the bottom of the page (this may take some time, so, be patient)
- **3)** Click on a point in the visualization tool. The interactive module will show the cartesian coordinates as well as the associated probability to failure of the point. 
- **4)** If the selected point is the correct point from which you wish to create a sub-model, click the button. This will create a new numpy file titles submodelCenterPoint.npy that consists of the cartesian coordinates of the selected point (three float values. the file will be very small, ~1kB). Note, this file will be overwritten if it exists. 

In [1]:
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import pandas as pd
#import plotly as py
import plotly.graph_objects as go
import os
from plotly.callbacks import Points, InputDeviceState
from ipywidgets import widgets
points, state = Points(), InputDeviceState()
plt.close('all')


def pointPrint(trace, points, state):
    global pointSave,misesPlot, elementCoord, xDisp, yDisp, zDisp
    ind=points.point_inds
    pointCoords=elementCoord[ind,:]
    xDisp.value=elementCoord[ind,0]#]pointCoords[0]
    yDisp.value=elementCoord[ind,1]#pointCoords[1]
    zDisp.value=elementCoord[ind,2]#pointCoords[2]
    probDisp.value=100*np.round(misesPlot[ind],3)

def buttonClick(b):
    global xDisp, yDisp, zDisp
    pointSave=np.zeros((3))
    pointSave[0]=xDisp.value
    pointSave[1]=yDisp.value
    pointSave[2]=zDisp.value
    np.save('submodelCenterPoint.npy',pointSave)

gateName='Greenup'

distro=np.load('DistroOneOverN.npy')
distro=1/distro
probability=np.zeros((distro.shape[0],1))
pointSave=[]
for i in range(distro.shape[0]):
    probability[i]=1-stats.gaussian_kde(distro[i,:]).integrate_box_1d(0,1)
    

nodes=pd.read_csv('nodesForVisualization.csv')

elements=pd.read_csv('misesAveragingMap.csv')

triangulation=np.load('triangulation.npy')
triangulationID=np.load('triangulationID.npy')
triangulationInstance=np.load('triangulationInstance.npy')

elementsTravis=np.load(gateName+'Elements_master.npy')
instancesTravis=np.load(gateName+'Instances_master.npy', allow_pickle=True)


misesPlot=np.zeros((triangulation.shape[0],1))
misesStrings=[None]*triangulation.shape[0]
#start=time.time()
names=np.unique(instancesTravis)


values={}
for name in names:
    values[name+'elementID']=elementsTravis[instancesTravis==name]
    values[name+'probability']=probability[instancesTravis==name]

for name in names:
    indexNames=np.argwhere(triangulationInstance==name)
    for elementID in range(len(values[name+'elementID'])):
        indexID=np.argwhere(triangulationID==values[name+'elementID'][elementID])
        indexSave=np.intersect1d(indexNames,indexID)
        misesPlot[indexSave]=values[name+'probability'][elementID]
elementCoord=np.zeros((len(misesStrings),3))
for i in range(len(misesStrings)):
    x1=nodes.x.iloc[triangulation[i,0]-1]
    x2=nodes.x.iloc[triangulation[i,1]-1]
    x3=nodes.x.iloc[triangulation[i,2]-1]
    y1=nodes.y.iloc[triangulation[i,0]-1]
    y2=nodes.y.iloc[triangulation[i,1]-1]
    y3=nodes.y.iloc[triangulation[i,2]-1]
    z1=nodes.z.iloc[triangulation[i,0]-1]
    z2=nodes.z.iloc[triangulation[i,1]-1]
    z3=nodes.z.iloc[triangulation[i,2]-1]
    elementCoord[i,0]=(x1+x2+x3)/3
    elementCoord[i,1]=(y1+y2+y3)/3
    elementCoord[i,2]=(z1+z2+z3)/3
    misesStrings[i]=str(np.round(100*misesPlot[i][0],4)) \
        + os.linesep+' x=' + \
            str(np.round((x1+x2+x3)/3,3)) +'\r\n y=' \
                + str(np.round((y1+y2+y3)/3,3)) \
                    +'\r\n z='+str(np.round((z1+z2+z3)/3,3))

trace=go.Mesh3d(x=nodes.x,y=nodes.y,z=nodes.z,
                colorscale=[[0, 'blue'],
                            [0.25, 'cyan'],
                            [0.5,'lightgreen'],
                            [0.75,'yellow'],
                            [1,'red']],
                i=triangulation[:,0],j=triangulation[:,1],k=triangulation[:,2], 
                intensity=misesPlot,
                intensitymode='cell',
                text=misesStrings,
                hoverinfo='text')

probDisp=widgets.FloatText(
value=0,
description='Probability',
disabled=True)

xDisp=widgets.FloatText(
value=0,
description='x',
disabled=True)

yDisp=widgets.FloatText(
value=0,
description='y',
disabled=True)

zDisp=widgets.FloatText(
value=0,
description='z',
disabled=True)

layout=widgets.Layout(width='auto',height='40px')
b1=widgets.Button(description='Click here to save this point',align_items='stretch',display='flex',layout=layout)
b1.on_click(buttonClick)
fig=go.Figure(data=[trace])#, layout=layout)
f2=go.FigureWidget(fig)
scatter=f2.data[0]
scatter.on_click(pointPrint)
widgets.VBox([widgets.HBox([probDisp,b1]),xDisp,yDisp,zDisp,f2])

